In [2]:

from pyspark.sql import SparkSession


spark = SparkSession.builder.appName("My Spark Application").getOrCreate()
df = spark.read.option("header", "true").csv("input.csv")
df.show(1000)


+--------------------+--------------------+--------------------+--------------------+
|            FILENAME|         ENTITY_TYPE|              STATUS|       ERROR_MESSAGE|
+--------------------+--------------------+--------------------+--------------------+
|transportInstruct...|transportInstruction|    INGESTION_FAILED|Error parsing AVR...|
|transportInstruct...|transportInstruction|DATA_INGESTION_ST...|                null|
+--------------------+--------------------+--------------------+--------------------+



In [11]:
df = spark.read.format("avro").load("inventoryMeasure_IKEA_7.avro")
#df.map(input -> input.itemId + 'xxx').show(1000)
df.show(10)

+------------------+--------+----------------+----------+---------------+--------------------+----------+
|documentActionCode|  itemId|ownerOfTradeItem|locationId|ownerOfLocation|             measure|extensions|
+------------------+--------+----------------+----------+---------------+--------------------+----------+
|              null|00246163|            IKEA|    STO189|           IKEA|[{Customer Order,...|      null|
|              null|00246163|            IKEA|    STO448|           IKEA|[{Customer Order,...|      null|
|              null|00246163|            IKEA|    STO886|           IKEA|[{Customer Order,...|      null|
|              null|00246361|            IKEA|    STO512|           IKEA|[{Customer Order,...|      null|
|              null|00246375|            IKEA|    STO392|           IKEA|[{Customer Order,...|      null|
|              null|00246375|            IKEA|    STO485|           IKEA|[{Customer Order,...|      null|
|              null|00247412|            IKEA|

In [5]:
#df.createOrReplaceTempView("inventoryMeasure")
#spark.sql("select  documentActionCode, extensions, itemId, locationId, explodedMeasure.inventoryMeasureCode, explodedmeasureValue[0].startDate, explodedmeasureValue[0].quantity, explodedmeasureValue[0].extensions as measureValueExtensions, ownerOfLocation, ownerOfTradeItem from inventoryMeasure lateral view explode(measure) as explodedMeasure lateral view explode(explodedMeasure) as explodedmeasureValue").show(10,50)

In [6]:
#spark.sql("insert into stg_inventoryMeasure select  documentActionCode, extensions, itemId, locationId, explodedMeasure.inventoryMeasureCode, measureValue.startDate, measureValue.quantity, measureValue.extensions as measureValueExtensions, ownerOfLocation, ownerOfTradeItem from inventoryMeasure lateral view explode(measure) as explodedMeasure lateral view explode(measure.measureValue) as measureValue")
#spark.sql("select * from stg_inventoryMeasure").show()

In [7]:
numbers = spark.range(1000).toDF("number")
numbers.show(10)

+------+
|number|
+------+
|     0|
|     1|
|     2|
|     3|
|     4|
|     5|
|     6|
|     7|
|     8|
|     9|
+------+
only showing top 10 rows



In [8]:
numbers.where("number % 2 = 0").show(10)

+------+
|number|
+------+
|     0|
|     2|
|     4|
|     6|
|     8|
|    10|
|    12|
|    14|
|    16|
|    18|
+------+
only showing top 10 rows



In [48]:
from pyspark.sql.functions import desc
flight_data_2015 = spark.read.option("inferSchem", "true").option("header", "true").csv("flight-data/csv/2015-summary.csv")
flight_data_2015.sort(desc("count")).show(100)

flight_data_2015.printSchema()

+--------------------+--------------------+------+
|   DEST_COUNTRY_NAME| ORIGIN_COUNTRY_NAME| count|
+--------------------+--------------------+------+
|       United States|         The Bahamas|   986|
|         The Bahamas|       United States|   955|
|       United States|              France|   952|
|              France|       United States|   935|
|       United States|               China|   920|
|             Curacao|       United States|    90|
|            Colombia|       United States|   873|
|       United States|            Colombia|   867|
|              Brazil|       United States|   853|
|       United States|              Canada|  8483|
|              Canada|       United States|  8399|
|        Saudi Arabia|       United States|    83|
|       United States|             Curacao|    83|
|       United States|         South Korea|   827|
|       United States|British Virgin Is...|    80|
|         Netherlands|       United States|   776|
|               China|       Un

In [31]:
flight_data_2015.createOrReplaceTempView("flight_data_2015")
spark.sql("select dest_country_name, count(1) from flight_data_2015 group by dest_country_name order by count(1) desc").show(100)

+--------------------+--------+
|   dest_country_name|count(1)|
+--------------------+--------+
|       United States|     125|
|             Germany|       1|
|                Iraq|       1|
|              Turkey|       1|
|            Anguilla|       1|
|              Russia|       1|
|            Paraguay|       1|
|               Palau|       1|
|                Fiji|       1|
|            Malaysia|       1|
|           Singapore|       1|
|              Jordan|       1|
|         Philippines|       1|
|             Senegal|       1|
|              Greece|       1|
|            Djibouti|       1|
|Turks and Caicos ...|       1|
|              France|       1|
|            Kiribati|       1|
|              Kosovo|       1|
|             Finland|       1|
|           Nicaragua|       1|
|               Ghana|       1|
|              Guyana|       1|
|              Taiwan|       1|
|British Virgin Is...|       1|
|            Dominica|       1|
|       New Caledonia|       1|
|       

In [34]:
flight_data_2015.groupBy("dest_country_name").count().orderBy(desc("count")).show(100)

+--------------------+-----+
|   dest_country_name|count|
+--------------------+-----+
|       United States|  125|
|              Turkey|    1|
|             Senegal|    1|
|         Philippines|    1|
|              Greece|    1|
|            Anguilla|    1|
|              Russia|    1|
|            Paraguay|    1|
|                Iraq|    1|
|              Jordan|    1|
|Turks and Caicos ...|    1|
|              France|    1|
|             Germany|    1|
|            Djibouti|    1|
|               Palau|    1|
|            Kiribati|    1|
|              Sweden|    1|
|              Guyana|    1|
|            Malaysia|    1|
|           Singapore|    1|
|                Fiji|    1|
|              Kosovo|    1|
|              Taiwan|    1|
|British Virgin Is...|    1|
|            Dominica|    1|
|             Belgium|    1|
|              Angola|    1|
|               India|    1|
|             Curacao|    1|
|               China|    1|
|          Cape Verde|    1|
|    Marshall 

In [44]:
flight_data_2015.select("count").orderBy(desc("count")).show(1)

+-----+
|count|
+-----+
|  986|
+-----+
only showing top 1 row



In [49]:
flight_data_2015.withColumn("countAsInt","count").cast(IntegerType()).groupBy("dest_country_name").sum("countAsInt").withColumnRenamed("sum(countAsInt)" , "dest_country_count").sort(desc("dest_country_count")).show(5)

AssertionError: col should be Column